# Auto-MPG
---

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('datasets/auto-mpg.csv')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


---
### Pipe 1

In [3]:
def num_prefix(dataframe, str_column):
    
    # Apply function if the year is not in 19-- format    
    if (dataframe[str_column] < 1000).all():
        
        # Apply prefix concatenation to the number   
        dataframe[str_column] = dataframe[str_column].apply(lambda x: 1900 + x)
    
    else:
        pass
    
    return dataframe 

---
### Pipe 2

In [4]:
def first_word_extract(dataframe, column, new_colname):
    
    # Change data type into string
    dataframe[column] = dataframe[column].astype('str')
    
    # Split converted string and store it in a series
    col_data = pd.Series(dtype='object')
    col_data['phrase_aslist'] = dataframe[column].str.split()
    
    # Retrieve the first content in the splitted phrase contained as a list
    dataframe[new_colname] = col_data['phrase_aslist'].apply(lambda words_aslist: words_aslist[0])
    
    return dataframe 

---

## Pandas Pipeline

In [5]:
# Optional: View all of your created functions

import inspect

names = dir()

for name in names:
    obj = eval(name)
    if inspect.isfunction(obj):
        print(name)

first_word_extract
num_prefix


In [6]:
piped_df = (df.pipe(num_prefix, 'model year')
              .pipe(first_word_extract, 'car name', 'car brand')
           )

In [7]:
piped_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,car brand
0,18.0,8,307.0,130,3504,12.0,1970,1,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,1970,1,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,1970,1,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,1970,1,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,1970,1,ford torino,ford
